In [ ]:

from flask import Flask, request, jsonify
import pandas as pd

app = Flask(__name__)

def analyze_financial_data():
    df = pd.read_csv('path_to_your_csv_file.csv')
    df['Revenue Growth (%)'] = df.groupby(['Company'])['Total Revenue'].pct_change() * 100
    df['Net Income Growth (%)'] = df.groupby(['Company'])['Net Income'].pct_change() * 100
    df['Assets Growth (%)'] = df.groupby(['Company'])['Total Assets'].pct_change() * 100
    df['Liabilities Growth (%)'] = df.groupby(['Company'])['Total Liabilities'].pct_change() * 100
    df['Cash Flow Growth (%)'] = df.groupby(['Company'])['Cash Flow from Operating Activities'].pct_change() * 100
    summary = df.groupby('Company')[['Revenue Growth (%)', 'Net Income Growth (%)',
                                     'Assets Growth (%)', 'Liabilities Growth (%)',
                                     'Cash Flow Growth (%)']].mean()
    df.to_csv('updated_financial_data.csv', index=False)
    return df, summary

data, summary = analyze_financial_data()

@app.route('/chat', methods=['POST'])
def chatbot():
    user_query = request.json.get('query', '').strip().lower()
    if "total revenue" in user_query:
        response = data.groupby('Company')["Total Revenue"].sum().to_dict()
    elif "net income growth" in user_query:
        response = summary["Net Income Growth (%)"].to_dict()
    elif "assets growth" in user_query:
        response = summary["Assets Growth (%)"].to_dict()
    elif "liabilities growth" in user_query:
        response = summary["Liabilities Growth (%)"].to_dict()
    elif "cash flow growth" in user_query:
        response = summary["Cash Flow Growth (%)"].to_dict()
    else:
        response = "Sorry, I can only provide information on predefined queries."
    return jsonify(response)

if __name__ == '__main__':
    app.run(debug=True)